In [1]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from collections import Counter
from tqdm.auto import tqdm

cuda


### swinv2-T

In [2]:
from networks import swinv2_base_patch4_window8_256, pretrained_swinv2_tiny_patch4_window8_256
from dataloader import *

In [ ]:
model = pretrained_swinv2_tiny_patch4_window8_256()
model.eval()
model.to(device)

/home/ray/proj/myenv/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=(64, 64), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=(64, 64), num_heads=3, window_size=8, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            dim=96, window_size=(8, 8), pretrained_window_size=(0, 0), num_heads=3
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=3, bias=False)
            )
            (qkv): Linear(in_features=96, out_features=288, bias=False)
            (attn_drop): Dropo

In [4]:
dataset_val, _= build_imagenet_val_dataset(input_size=256)
imagnet_dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=1,
    num_workers = 0,
)

Transform = 
Resize(size=292, interpolation=bicubic, max_size=None, antialias=None)
CenterCrop(size=(256, 256))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet
Number of the class = 1000


In [7]:
def validate(model, dataloader):
    correct_top1 = 0
    total_samples = 0
    accumulated_label = 0
    accumulated_predictions = []

    with torch.no_grad():
        for input, label in tqdm(dataloader):

            input, label = input.to(device), label.to(device)

            # Forward pass
            output = model(input)

            # Get top-1 predictions
            _, pred_top1 = output.topk(1, dim=1, largest=True, sorted=True)
            
            curr_label = label.item()
            if curr_label==accumulated_label:
                accumulated_predictions.append(pred_top1.item())
            else:
                counter = Counter(accumulated_predictions)
                _, count = counter.most_common(1)[0]
                total_samples += len(accumulated_predictions)
                correct_top1 += count
                accumulated_label = curr_label
                accumulated_predictions = []
    counter = Counter(accumulated_predictions)
    _, count = counter.most_common(1)[0]
    total_samples += len(accumulated_predictions)
    correct_top1 += count
    # Calculate top-1 accuracy percentage
    top1_accuracy = 100 * correct_top1 / total_samples
    return top1_accuracy

top1_accuracy = validate(model, imagnet_dataloader_val)
print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')

  0%|          | 0/50000 [00:00<?, ?it/s]

Top-1 Accuracy: 81.92%


Claimed Accuracy: 81.8%

Tested Accuracy: 81.92%

### swinv2-B

In [12]:
from networks import swinv2_base_patch4_window8_256, pretrained_swinv2_base_patch4_window8_256
from dataloader import *

In [13]:
model = pretrained_swinv2_base_patch4_window8_256()
model.eval()
model.to(device)

SwinTransformerV2(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=128, input_resolution=(64, 64), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=128, input_resolution=(64, 64), num_heads=4, window_size=8, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            dim=128, window_size=(8, 8), pretrained_window_size=(0, 0), num_heads=4
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
            (qkv): Linear(in_features=128, out_features=384, bias=False)
            (attn_drop)

In [14]:
dataset_val, _= build_imagenet_val_dataset(input_size=256)
imagnet_dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=1,
    num_workers = 0,
)

Transform = 
Resize(size=292, interpolation=bicubic, max_size=None, antialias=None)
CenterCrop(size=(256, 256))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet
Number of the class = 1000


In [15]:
def validate(model, dataloader):
    correct_top1 = 0
    total_samples = 0
    accumulated_label = 0
    accumulated_predictions = []

    with torch.no_grad():
        for input, label in tqdm(dataloader):

            input, label = input.to(device), label.to(device)

            # Forward pass
            output = model(input)

            # Get top-1 predictions
            _, pred_top1 = output.topk(1, dim=1, largest=True, sorted=True)
            
            curr_label = label.item()
            if curr_label==accumulated_label:
                accumulated_predictions.append(pred_top1.item())
            else:
                counter = Counter(accumulated_predictions)
                _, count = counter.most_common(1)[0]
                total_samples += len(accumulated_predictions)
                correct_top1 += count
                accumulated_label = curr_label
                accumulated_predictions = []
    counter = Counter(accumulated_predictions)
    _, count = counter.most_common(1)[0]
    total_samples += len(accumulated_predictions)
    correct_top1 += count
    # Calculate top-1 accuracy percentage
    top1_accuracy = 100 * correct_top1 / total_samples
    return top1_accuracy

top1_accuracy = validate(model, imagnet_dataloader_val)
print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')

  0%|          | 0/50000 [00:00<?, ?it/s]

Top-1 Accuracy: 84.40%


Claimed Accuracy: 84.2%

Tested Accuracy: 84.40%